# **Download ISIC**

In [ ]:
! wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Training_Input.zip
! unzip ISIC2018_Task3_Training_Input.zip
! wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Validation_Input.zip
! unzip ISIC2018_Task3_Validation_Input.zip

# **Classification Libs**

In [2]:
# imports
import glob
import cv2
import os
import random
import copy
import time
import numpy as np
from google.colab.patches import cv2_imshow
import pandas as pd

import torch
import torch.nn as nn
from tqdm.auto import tqdm, trange
from torchvision import datasets, models, transforms
import torch.optim as optim
from torchvision import transforms

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedShuffleSplit

from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.metrics import peak_signal_noise_ratio
from skimage.util import random_noise

# scripts import
# from scripts import * # ????

torch.cuda.empty_cache()
random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# CUDA_LAUNCH_BLOCKING="1"

device(type='cuda')

# **Classification Funcs**

In [3]:
def read_dataset(path):
    # dataset = []
    # for filename in tqdm(glob.iglob(path + '/*.png' , recursive=True) , desc = "Reading"):
    #     # image_name = filename.split('/')[1] 
    #     # class_type = image_name.split('_')[0]
    #     image_name = filename;
    #     image = cv2.imread(filename)
    #     image =  transforms.ToPILImage()(image)
    #     image = transforms.Resize((224 , 224))(image)
    #     image = transforms.ToTensor()(image)
    #     dataset.append((image , int(class_type)))
    # return dataset     
    dataset = []
    df = pd.read_csv (path+'_GroundTruth.csv')
    df2 = pd.DataFrame(df, columns= ['image'])
    df2 = df2.to_numpy();
    df = pd.DataFrame(df, columns= ['MEL','NV','BCC','AKIEC','BKL','DF','VASC'])
    df = df.to_numpy();
    # if(len(df)>200):
    #   df = df[1:5000];
    for i,j in enumerate(df):
      # print((i,j))
      class_type = np.where(j)[0][0];
      # print(class_type)
      image = cv2.imread(path+'_Input/'+df2[i][0]+'.jpg')
      # print(image.shape)
      for t in range(3):
        mi = image[:,:,t].min();
        ma = image[:,:,t].max();
        image[:,:,t] = (image[:,:,t]-mi)/(ma-mi)*255;
        # mi = image[:,:,t].min();
        # ma = image[:,:,t].max();
        # print([mi, ma]);
      image = cv2.resize(image, (224, 224));
      # cv2_imshow(image)
      # image = NLMDenoiser(image); # Denoising
      # cv2_imshow(NLMDenoiser(image))
      image =  transforms.ToPILImage()(image)
      # image = transforms.Resize((299 , 299))(image)
      # image = transforms.Resize((224 , 224))(image)
      image = transforms.ToTensor()(image)
      dataset.append((image , int(class_type)))
      # break;
    return dataset 

def read_dataset_transformation_all(path): 
    dataset = []
    df = pd.read_csv (path+'_GroundTruth.csv')
    df2 = pd.DataFrame(df, columns= ['image'])
    df2 = df2.to_numpy();
    df = pd.DataFrame(df, columns= ['MEL','NV','BCC','AKIEC','BKL','DF','VASC'])
    df = df.to_numpy();
    for i,j in enumerate(df):
      class_type = np.where(j)[0][0];
      image = cv2.imread(path+'_Input/'+df2[i][0]+'.jpg')
      for t in range(3):
        mi = image[:,:,t].min();
        ma = image[:,:,t].max();
        image[:,:,t] = (image[:,:,t]-mi)/(ma-mi)*255;
      image =  transforms.ToPILImage()(image)
      image = transforms.Resize((224 , 224))(image)
      image = transforms.RandomHorizontalFlip()(image)
      image = transforms.RandomAffine(degrees = 90, translate= (0.2 , 0.2))(image)
      image = transforms.ToTensor()(image)
      dataset.append((image , int(class_type)))
    return dataset 

def transformation_all(dataset):
    new_dataset = []
    for image , label in dataset: 
        image = transforms.ToPILImage()(image)
        # image = transforms.RandomHorizontalFlip()(image)
        image = transforms.functional.hflip(image)
        # image = transforms.RandomAffine(degrees = 90, translate= (0.2 , 0.2))(image)
        image = transforms.ToTensor()(image)
        new_dataset.append((image , label))
    return new_dataset

def NLMDenoiser(noisy):
  # noisy = transforms.ToPILImage()(noisy)
  # sigma_est = np.mean(estimate_sigma(noisy, channel_axis=-1))
  # sigma_est = np.mean(estimate_sigma(noisy))
  # patch_kw = dict(patch_size=500,      # 5x5 patches
  #               patch_distance=600,  # 13x13 search area
  #             )
  #               # channel_axis=-1)
  # denoise2_fast = denoise_nl_means(noisy, h=0.6 * sigma_est, sigma=sigma_est,
  #                                fast_mode=True, **patch_kw)
  # denoise2_fast = transforms.ToTensor()(denoise2_fast)
  
  # cv2_imshow(noisy)
  # noisy = [np.uint8(np.clip(i,0,255)) for i in noisy]
  # denoise2_fast = cv2.fastNlMeansDenoisingMulti(noisy, 2, 1, None, 4, 7, 35);
  denoise2_fast = cv2.fastNlMeansDenoisingColored(noisy,None,10,10,7,21)
  # cv2_imshow(denoise2_fast)
  return denoise2_fast;

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
    if feature_extracting == False:
        for param in model.parameters():
            param.requires_grad = True

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224


    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft

def train_model(model, dataloaders, criterion, optimizer, scheduler = False, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.2 * loss2
                    else:
                        outputs = model(inputs)
                        # print(outputs.shape)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        # scheduler.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            if(phase=='val'):
              scheduler.step(epoch_loss)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    # Scheduler
    # Hyperparameters
    # load best model weights
    model.load_state_dict(best_model_wts)
    PATH = "state_dict_model.pt"
    torch.save(model.state_dict(), PATH)
    return model, val_acc_history


# **Read Dataset**

In [4]:
csv_path = '';

train_dataset = read_dataset('ISIC2018_Task3_Training')

val_dataset = read_dataset('ISIC2018_Task3_Validation')

# train_dataset_augmented = transformation_all(train_dataset);
# train_dataset = train_dataset_augmented + train_dataset 

print(len(train_dataset))
print(len(val_dataset))

10015
193


# **Train**

In [ ]:
print(train_dataset[0][0].shape)
print(len(train_dataset))
print(train_dataset[0])
print(next(iter(train_loader)))

In [5]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet"

# Number of classes in the dataset
num_classes = 7

# Batch size for training (change depending on how much memory you have)
BATCH_SIZE = 5

# Number of epochs to train for
num_epochs = 50

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = False

# Initialize the model for this run
model = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Send the model to GPU
model = model.to(device)

# min max normalization
# augmentation

optimizer = optim.Adam(model.parameters(), lr=1e-4)
# optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
# optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
# lambda1 = lambda epoch: 0.65 ** epoch
# scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)
# optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=10, threshold=1e-8, verbose=True)

# optimized = SGD
# optimized = SGD Momentum = 0.9
# scheduler

criterion = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=True)
dataloaders_dict = {'train' : train_loader , 'val': val_loader }


model , val_acc = train_model(model, dataloaders_dict, criterion, optimizer, scheduler , num_epochs = num_epochs , is_inception = (model_name=="inception"))


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Epoch 0/49
----------
train Loss: 0.8184 Acc: 0.7123
val Loss: 0.7335 Acc: 0.7565

Epoch 1/49
----------
train Loss: 0.6777 Acc: 0.7559
val Loss: 0.6763 Acc: 0.7461

Epoch 2/49
----------
train Loss: 0.5752 Acc: 0.7945
val Loss: 0.5785 Acc: 0.7876

Epoch 3/49
----------
train Loss: 0.4499 Acc: 0.8401
val Loss: 0.5925 Acc: 0.7927

Epoch 4/49
----------
train Loss: 0.3296 Acc: 0.8850
val Loss: 0.7717 Acc: 0.7772

Epoch 5/49
----------
train Loss: 0.2272 Acc: 0.9221
val Loss: 1.1179 Acc: 0.7202

Epoch 6/49
----------
train Loss: 0.1614 Acc: 0.9460
val Loss: 0.7719 Acc: 0.7824

Epoch 7/49
----------
train Loss: 0.1245 Acc: 0.9587
val Loss: 0.9619 Acc: 0.7668

Epoch 8/49
----------
train Loss: 0.0997 Acc: 0.9686
val Loss: 0.8780 Acc: 0.7824

Epoch 9/49
----------
train Loss: 0.0975 Acc: 0.9671
val Loss: 1.1595 Acc: 0.7876

Epoch 10/49
----------
train Loss: 0.0825 Acc: 0.9741
val Loss: 0.8809 Acc: 0.7617

Epoch 11/49
----------
train Loss: 0.0700 Acc: 0.9784
val Loss: 1.3246 Acc: 0.7150

Ep

# **Save Model**

In [ ]:
torch.save(model, 'Model.pt')

# **Load Model**

Restart runtime first.

In [ ]:
val_dataset = transformation_all(val_dataset)

In [ ]:
model = torch.load('Model.pt')
csv_path = '';

train_dataset = read_dataset_transformation_all('ISIC2018_Task3_Training')

val_dataset = read_dataset('ISIC2018_Task3_Validation')

print(len(train_dataset))
print(len(val_dataset))


10015
193


In [ ]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet"
num_classes = 7
BATCH_SIZE = 5
num_epochs = 50
feature_extract = False

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=10, threshold=1e-8, verbose=True)
criterion = nn.CrossEntropyLoss()

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=True)
dataloaders_dict = {'train' : train_loader , 'val': val_loader }

model , val_acc = train_model(model, dataloaders_dict, criterion, optimizer, scheduler , num_epochs = num_epochs , is_inception = (model_name=="inception"))


Epoch 0/49
----------
train Loss: 0.6661 Acc: 0.7861
val Loss: 0.5809 Acc: 0.8187

Epoch 1/49
----------
train Loss: 0.3970 Acc: 0.8622
val Loss: 0.5510 Acc: 0.8342

Epoch 2/49
----------
train Loss: 0.2333 Acc: 0.9222
val Loss: 0.6923 Acc: 0.8497

Epoch 3/49
----------
train Loss: 0.1317 Acc: 0.9571
val Loss: 0.9683 Acc: 0.7772

Epoch 4/49
----------
train Loss: 0.0914 Acc: 0.9721
val Loss: 1.0680 Acc: 0.8135

Epoch 5/49
----------
train Loss: 0.0807 Acc: 0.9743
val Loss: 0.9495 Acc: 0.8601

Epoch 6/49
----------
train Loss: 0.0732 Acc: 0.9800
val Loss: 0.5922 Acc: 0.8446

Epoch 7/49
----------
train Loss: 0.0748 Acc: 0.9769
val Loss: 0.7485 Acc: 0.8187

Epoch 8/49
----------
train Loss: 0.0498 Acc: 0.9832
val Loss: 0.9438 Acc: 0.8342

Epoch 9/49
----------
train Loss: 0.0531 Acc: 0.9837
val Loss: 0.9222 Acc: 0.8549

Epoch 10/49
----------
train Loss: 0.0456 Acc: 0.9851
val Loss: 1.0862 Acc: 0.7979

Epoch 11/49
----------
train Loss: 0.0537 Acc: 0.9830
val Loss: 0.8385 Acc: 0.8497

Ep